In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader

In [1]:
import numpy as np
import pandas as pd
import os
import json
import csv
import cv2
import seaborn as sn
import matplotlib.pyplot as plt

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_16012\640182863.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
def load_listing(directory: str):
    data = []
    for i in range (16):
        index = hex(i)[2:]
        file_name = "listings_" + index + ".json"
        path = os.path.join(directory, file_name)
        
        #english_speaking_countries = {"US", "GB", "CA", "AU"}
        english_speaking_countries = {"US", "GB", "CA", "AU"}
        
        with open(path, "r", encoding='utf-8') as f:
            for line in f:
                line = line.strip()
                if line:
                    obj = json.loads(line)
                    if obj["country"] in english_speaking_countries:
                        data.append(obj)
    return data

In [3]:
data = load_listing(r"D:\Code\VS Code\AI_project\Product_Classification\images\metadata\listing")

In [ ]:
def extract_main_category(listings):
    main_categories = set()
    for item in listings:
        nodes = item.get("node", [])
        if nodes:
            for node in nodes:
                node_name = node.get("node_name", "")
                if node_name:
                    parts = node_name.split('/')
                    if len(parts) > 2:
                        if (parts[1] != "Categories"):
                            main_category = parts[1]
                        else:
                            main_category = parts[2]
                        main_categories.add(node_name)
    return main_categories

In [16]:
print(f"Loaded {len(data)} JSON objects")
categories = extract_main_category(data)
categories = list(set(categories))
categories.sort()

Loaded 39137 JSON objects


In [17]:
print(len(categories))
for category in categories:
    print(category)

4560
/Archive/2020-09/5488469050 - Sweeping Cloths & Wipes
/Archive/2020-11/3662063030 - Moisturising & Rejuvenating Masks
/Archive/2020-12/1174994030 - Protective Flooring
/Archive/2020-12/2078321030 - Mains Chargers
/Archive/2020-12/5518611050 - Protective Flooring
/Archive/2021-03/7889891010 - Pendants
/Categories/3D Printing & Scanning/3D Printer Parts & Accessories/3D Printer Accessories
/Categories/3D Printing & Scanning/3D Printing Materials/Filament 3D Printing Materials
/Categories/Abrasive & Finishing Products/Abrasive Wheels & Discs/Cut-Off Wheels
/Categories/Accessories
/Categories/Accessories & Supplies/Audio & Video Accessories
/Categories/Accessories & Supplies/Audio & Video Accessories/Antennas/TV Antennas
/Categories/Accessories & Supplies/Audio & Video Accessories/Cables & Interconnects
/Categories/Accessories & Supplies/Audio & Video Accessories/Cables & Interconnects/Audio Cables
/Categories/Accessories & Supplies/Audio & Video Accessories/Cables & Interconnects/Vid

In [89]:
category_name = "Sports Apparel"

for listing in data:
    nodes = listing.get("node")
    if not nodes or not isinstance(nodes, list) or not nodes[0]:
        continue

    node_name = nodes[0].get("node_name")
    if not node_name:
        continue

    parts = node_name.split('/')
    if len(parts) > 2:
        if parts[1] != "Categories":
            if parts[1] == category_name:
                print(listing["main_image_id"])
        else:
            if parts[2] == category_name:
                print(listing)


{'brand': [{'language_tag': 'en_CA', 'value': 'CGTL'}], 'color': [{'language_tag': 'en_CA', 'value': '1 Pair Gray/White'}], 'item_id': 'B08GFHBMNT', 'item_name': [{'language_tag': 'en_CA', 'value': 'Warm Thermal Socks, CGTL Unisex Fur Lined Winter Skiing Socks Thick Boot Insulated Heated Socks for Cold Weather 1 Pack'}], 'model_number': [{'value': 'LQV01LMWZY0102M'}], 'product_type': [{'value': 'HAT'}], 'main_image_id': '71uwVpjrX2L', 'other_image_id': ['71rC0VlnsbL'], 'country': 'CA', 'marketplace': 'Amazon', 'domain_name': 'amazon.ca', 'node': [{'node_id': 2439792011, 'node_name': '/Categories/Sports Apparel/Men/Skiing/Socks'}, {'node_id': 8794302011, 'node_name': '/Categories/Men/Clothing/Activewear/Socks'}, {'node_id': 10287312011, 'node_name': '/Categories/Men/Clothing/Socks & Tights'}]}
{'brand': [{'language_tag': 'en_CA', 'value': 'BINACL'}], 'color': [{'language_tag': 'en_CA', 'value': 'Style02-hemp Gray-1 Pack'}], 'item_id': 'B08CH8KVSX', 'item_name': [{'language_tag': 'en_CA'

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = models.resnet50(pretrained=True)

# Freeze all layers
for param in model.parameters():
    param.requires_grad = False

# Replace final layer
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(class_names))  # 20 classes

model = model.to(device)
